Módulo 01: Descarga y Preparación de Datos
### Versión: `01_data_v1.4.4` – Fecha: 2025-10-20
**Estado:** Producción estable  
**Autor:** W. Aguirre — *Proyecto Mini Reinanse (Sistema Cuantitativo Modular)*  

---

## 📘 Descripción General
Este módulo descarga, valida y prepara los datos históricos de precios para todos los activos bursátiles disponibles en MetaTrader 5 (CFDs de acciones US).  
Los datos se transforman en series temporales limpias y se enriquecen con indicadores técnicos y métricas de calidad para ser utilizados en los módulos posteriores de modelado, backtesting y ejecución.

---

## ⚙️ Flujo General de Proceso
1. **Inicialización y configuración de rutas**
2. **Conexión a MetaTrader 5** (verificación de cuenta y servidor)
3. **Generación del universo de símbolos US** (`symbol_aliases.json`)
4. **Descarga paralela de datos históricos**
5. **Cálculo de features técnicos:**
   - Retornos (`returns`)
   - Volatilidad 20d (`volatilidad`)
   - Drawdown acumulado (`drawdown`)
   - Media móvil 20d (`ma20`)
   - Score compuesto (momentum / volatilidad)
   - Sharpe 20d y Momentum 63d
6. **Validaciones automáticas:**
   - Precios > 15 USD
   - Mínimo 250 barras
   - Sin huecos > 5 días
   - Volatilidad mínima > 1e-3
7. **Consolidación final**
   - `prices_df_diario.pkl`  →  Precios diarios
   - `prices_df.pkl`  →  Último precio mensual

---

## 📊 Salidas Principales
| Archivo | Descripción | Ubicación |
|----------|--------------|-----------|
| `symbol_aliases.json` | Mapeo simplificado alias ↔ ticker | `/notebooks/` |
| `symbol_aliases_ext.json` | Mapeo extendido con exchange y path | `/notebooks/` |
| `data/raw/*.csv` | Datos OHLC crudos por activo | `/data/raw/` |
| `data/processed/features_estado_*.csv` | Datos procesados + features | `/data/processed/` |
| `backtesting/prices_df_diario.pkl` | Precios diarios consolidados | `/backtesting/` |
| `backtesting/prices_df.pkl` | Precios mensuales (último de cada mes) | `/backtesting/` |
| `log_resumen_validacion.txt` | Log de integridad global | `/Mini reinanse/` |

---

## 🔍 Métricas de Integridad
- Activos procesados correctamente: **≈ 97 %**
- Motivos de descarte: `precio_bajo`, `datos_insuficientes`
- Integridad temporal y de features > 95 %
- Sin errores de `DatetimeIndex` ni duplicados

---

## 🧠 Mejoras de la Versión 1.4.4
| Categoría | Mejora |
|------------|---------|
| **Rendimiento** | Ejecución paralela optimizada (`ThreadPoolExecutor` + `tqdm`) |
| **Estabilidad** | Control de salida (preview limitado, sin saturar Jupyter) |
| **Compatibilidad** | Logging UTF-8 seguro (sin errores Unicode Windows) |
| **Validación** | Nuevas métricas de integridad y diagnóstico final |
| **Estructura** | Separación modular en 9 celdas (config → diagnóstico) |
| **Consolidación** | Creación de `.pkl` diarios y mensuales verificables |

---

## 🔒 Dependencias Principales
- `MetaTrader5`
- `pandas ≥ 1.5`
- `numpy`
- `tqdm`
- `concurrent.futures`
- `logging`
- `os`, `json`, `datetime`

---

## ✅ Estado Final del Módulo
| Componente | Estado |
|-------------|--------|
| Conexión MT5 | 🟢 Operativa |
| Descarga de datos | 🟢 Verificada |
| Cálculo de features | 🟢 Correcto |
| Validación temporal | 🟢 Sin huecos |
| Consolidación | 🟢 Completada |
| Integridad global | 🟢 97 % OK |

---

> Este notebook marca el punto de partida del pipeline cuantitativo de Mini Reinanse.  
> Los archivos generados son la base de entrada para `02_model.ipynb` (entrenamiento HMM/ML) y los módulos de backtesting y riesgo.


In [1]:
# 1.- Inicialización 
import re
import numpy as np
import pandas as pd

def detectar_bolsa(path: str) -> str:
    p = (path or "").lower()
    if "nasdaq" in p: return "NASDAQ"
    if "nyse" in p: return "NYSE"
    if "dow" in p: return "DOW"
    return "OTHER"

def safe_symbol_name(name: str) -> str:
    return re.sub(r'[^A-Za-z0-9\.\-_]+', '_', name).strip("_")

def alias_from_mt5_name(sym_name: str, path: str) -> str:
    m = re.search(r'\(([A-Z0-9\.\-]+)\)\s*$', sym_name)
    if m:
        return f"{m.group(1)}.US"
    last_seg = (path or sym_name).split("\\")[-1]
    last_seg = re.sub(r'[^A-Z0-9\.\-]+', '', last_seg, flags=re.I)
    if last_seg:
        return f"{last_seg}.US"
    base = re.sub(r'[#]+', '', sym_name)
    base = re.sub(r'(_i|_m|_ar|_x|\.NQ|\.NAS|\.NYSE)$', '', base, flags=re.I)
    return f"{base}.US"

def validar_huecos(index: pd.DatetimeIndex, max_gap_days=5) -> bool:
    if len(index) < 2:
        return False
    ordered = index.sort_values().asi8
    if ordered.size < 2:
        return False
    gaps = np.diff(ordered)
    max_gap = gaps.max() if gaps.size else 0
    return max_gap <= pd.Timedelta(days=max_gap_days).value

print("✅ Inicialización y configuración de rutas cargados correctamente.")


✅ Inicialización y configuración de rutas cargados correctamente.


In [2]:
# 2.- Configuración de rutas
import MetaTrader5 as mt5
import os, json, logging, re, time
from datetime import datetime, timedelta
from collections import defaultdict, OrderedDict
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

from hedgefund_paths import build_paths, ensure_directories, configure_logger

paths = build_paths()
BASE_DIR = paths['base']
PROJECT_DIR = paths['project']
NOTEBOOKS_DIR = paths['notebooks']
DATA_DIR = paths['data']
DATA_RAW_DIR = paths['data_raw']
DATA_PROCESSED_DIR = paths['data_processed']
ESTADO_DIR = paths['estado']
BACKTESTING_DIR = paths['backtesting']

ensure_directories([NOTEBOOKS_DIR, DATA_RAW_DIR, DATA_PROCESSED_DIR, ESTADO_DIR, BACKTESTING_DIR])

VERSION = "01_data_v1.4.4_2025-10-20"
LOG_PATH = NOTEBOOKS_DIR / "01_log_features_estado.log"
logger = configure_logger(LOG_PATH)
logger.info("Celda 1 ejecutada correctamente.")
logger.info("Directorio base: %s", PROJECT_DIR)

print("✅ Configuración de rutas creada correctamente.")
print(f"📁 Directorio base: {PROJECT_DIR}")

INFO - Celda 1 ejecutada correctamente.
INFO - Directorio base: C:\Users\User\Downloads\Mini reinanse\notebooks


✅ Configuración de rutas creada correctamente.
📁 Directorio base: C:\Users\User\Downloads\Mini reinanse\notebooks


In [3]:
# 3.- Parámetros de operacion
DIAS_HISTORICOS = 1095
TIMEFRAME = mt5.TIMEFRAME_D1
FECHA_HASTA = datetime.now() - timedelta(days=1)
FECHA_DESDE = FECHA_HASTA - timedelta(days=DIAS_HISTORICOS)
PRECIO_MINIMO = 15
VOL_MIN_MEDIA = 1e-3
HUECO_MAX_DIAS = 5
MIN_BARRAS = 250
MAX_WORKERS = min(6, (os.cpu_count() or 6))
REINTENTOS = 2
SLEEP_ENTRE_INTENTOS = 0.5

PATH_ALIASES = os.path.join(NOTEBOOKS_DIR, "symbol_aliases.json")
PATH_ALIASES_EXT = os.path.join(NOTEBOOKS_DIR, "symbol_aliases_ext.json")

print("📡 Intentando conexión con MetaTrader 5...")
if not mt5.initialize():
    raise SystemExit(f"🛑 No se pudo conectar a MetaTrader 5: {mt5.last_error()}")
acc = mt5.account_info()
print(f"✅ Conectado a cuenta {acc.login if acc else 'N/A'} en servidor {acc.server if acc else 'N/A'}")
print(f"📆 Rango de datos: {FECHA_DESDE.date()} → {FECHA_HASTA.date()}")


📡 Intentando conexión con MetaTrader 5...
✅ Conectado a cuenta 3000086292 en servidor Darwinex-Demo
📆 Rango de datos: 2022-11-17 → 2025-11-16


In [4]:
# 4.- Cálculo de indicadores técnicos
class FeatureCalculator:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()

    def add_returns(self):
        self.df["returns"] = self.df["close"].pct_change()
        return self

    def add_volatility(self, windows=[20, 63]):
        for w in windows:
            self.df[f"volatilidad_{w}"] = self.df["returns"].rolling(w).std()
        return self

    def add_drawdown(self, windows=[None]):
        for w in windows:
            if w is None:
                self.df["drawdown"] = self.df["close"] / self.df["close"].cummax() - 1
            else:
                self.df[f"drawdown_{w}"] = self.df["close"] / self.df["close"].rolling(w).max() - 1
        return self

    def add_moving_averages(self, windows=[20, 50, 200]):
        for w in windows:
            self.df[f"ma{w}"] = self.df["close"].rolling(w).mean()
        return self

    def add_zscore(self, window=20):
        ma = self.df["close"].rolling(window).mean()
        std = self.df["close"].rolling(window).std()
        self.df["zscore_close"] = (self.df["close"] - ma) / (std + 1e-8)
        return self

    def add_momentum(self, period=63):
        self.df[f"mom_{period}"] = self.df["close"].pct_change(period)
        return self

    def add_score_compuesto(self, window_returns=5, window_vol=20):
        avg_return = self.df["returns"].rolling(window_returns).mean()
        vol = self.df[f"volatilidad_{window_vol}"].replace(0, np.nan).fillna(1e-8)
        self.df["score_compuesto"] = avg_return / vol
        return self

    def add_sharpe(self, window=20):
        avg_return = self.df["returns"].rolling(window).mean()
        vol = self.df[f"volatilidad_{window}"].replace(0, np.nan).fillna(1e-8)
        self.df[f"sharpe_{window}"] = avg_return / vol
        return self

    def add_atr(self, window=14):
        tr1 = self.df["high"] - self.df["low"]
        tr2 = abs(self.df["high"] - self.df["close"].shift())
        tr3 = abs(self.df["low"] - self.df["close"].shift())
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        self.df[f"atr_{window}"] = tr.rolling(window).mean()
        return self

    def add_rsi(self, window=14):
        delta = self.df["close"].diff()
        up = delta.clip(lower=0)
        down = -delta.clip(upper=0)
        roll_up = up.rolling(window).mean()
        roll_down = down.rolling(window).mean()
        rs = roll_up / (roll_down + 1e-8)
        self.df[f"rsi_{window}"] = 100 - (100 / (1 + rs))
        return self

    def add_alpha_rank(self):
        mom = self.df["mom_63"]
        dd = self.df["drawdown"]
        self.df["alpha_rank"] = (
            0.5 * mom.rank(pct=True) +
            0.5 * (-dd).rank(pct=True)
        )
        return self

    def compute(self):
        return self.df

def calcular_features(df: pd.DataFrame) -> pd.DataFrame:
    return (
        FeatureCalculator(df)
        .add_returns()
        .add_volatility()
        .add_drawdown()
        .add_moving_averages()
        .add_zscore()
        .add_momentum()
        .add_score_compuesto()
        .add_sharpe()
        .add_atr()
        .add_rsi()
        .add_alpha_rank()
        .compute()
    )


In [5]:
# 5.- Universo de acciones
US_PATH_KEY = "stocks\\us"
all_syms = [s for s in mt5.symbols_get() if (s.path or "").lower().find(US_PATH_KEY) != -1]
tradables = [s for s in all_syms if (mt5.symbol_info(s.name) and mt5.symbol_info(s.name).trade_mode == mt5.SYMBOL_TRADE_MODE_FULL)]

print(f"📚 Detectados {len(all_syms)} símbolos US, de los cuales {len(tradables)} son operables.")

# ===== Mapping =====
mapping_simple = OrderedDict()
mapping_ext = OrderedDict()
for s in tradables:
    exch = detectar_bolsa(s.path)
    alias = alias_from_mt5_name(s.name, s.path)
    ticker_real = s.name
    mapping_simple[alias] = ticker_real
    mapping_ext[alias] = {"ticker": ticker_real, "exchange": exch, "path": s.path}

with open(PATH_ALIASES, "w", encoding="utf-8") as f:
    json.dump(mapping_simple, f, indent=2)
with open(PATH_ALIASES_EXT, "w", encoding="utf-8") as f:
    json.dump(mapping_ext, f, indent=2)

print(f"✅ Archivos de mapeo generados: {len(mapping_ext)} símbolos guardados.")
print("🔹 Ejemplo:", list(mapping_simple.items())[:5])

📚 Detectados 707 símbolos US, de los cuales 699 son operables.
✅ Archivos de mapeo generados: 699 símbolos guardados.
🔹 Ejemplo: [('A.US', 'A'), ('AA.US', 'AA'), ('AAP.US', 'AAP'), ('ABBV.US', 'ABBV'), ('AAL.US', 'AAL')]


In [6]:
# 6.- Exploracion de activos
import pandas as pd, os, json
print("🔍 Generando DataFrame con los primeros activos que se descargarán...")

df_universo = pd.DataFrame.from_dict(mapping_ext, orient="index").reset_index()
df_universo.rename(columns={"index": "alias"}, inplace=True)
df_universo = df_universo.sort_values("alias").reset_index(drop=True)

print(f"📦 Total detectados: {len(df_universo)} símbolos.")
print("🔸 Primeros 10 activos a descargar:")
display(df_universo.head(10))

PROJECT_DIR = "C:/Users/User/Downloads/Mini reinanse"
preview_path = os.path.join(PROJECT_DIR, "preview_descarga_mt5.csv")
df_universo.head(10).to_csv(preview_path, index=False, encoding="utf-8-sig")
print("Felicidades, se ha creado el archivo de vista previa (primeras 10 filas), útil para validar los datos cargados.")
print(f"✅ Vista previa guardada en: {preview_path}")

🔍 Generando DataFrame con los primeros activos que se descargarán...
📦 Total detectados: 699 símbolos.
🔸 Primeros 10 activos a descargar:


,alias,ticker,exchange,path
0,A.US,A,NYSE,Stocks\US\NYSE\A
1,AA.US,AA,NYSE,Stocks\US\NYSE\AA
2,AAL.US,AAL,NASDAQ,Stocks\US\Nasdaq\AAL
3,AAP.US,AAP,NYSE,Stocks\US\NYSE\AAP
4,AAPL.US,AAPL,DOW,Stocks\US\DOW\AAPL
5,ABBV.US,ABBV,NYSE,Stocks\US\NYSE\ABBV
6,ABT.US,ABT,NYSE,Stocks\US\NYSE\ABT
7,ACHC.US,ACHC,NASDAQ,Stocks\US\Nasdaq\ACHC
8,ACM.US,ACM,NYSE,Stocks\US\NYSE\ACM
9,ACN.US,ACN,NYSE,Stocks\US\NYSE\ACN


Felicidades, se ha creado el archivo de vista previa (primeras 10 filas), útil para validar los datos cargados.
✅ Vista previa guardada en: C:/Users/User/Downloads/Mini reinanse\preview_descarga_mt5.csv


In [7]:
# 7.- Descarga de datos de acciones
motivos_falla = defaultdict(list)
precios_consolidados = {}
activos_ok = []

# Control de preview para evitar saturar la salida
preview_counter = 0
preview_limit = 3

def descargar_barras(ticker_real: str):
    """Descarga barras desde MetaTrader 5 con reintentos automáticos."""
    rates = mt5.copy_rates_range(ticker_real, TIMEFRAME, FECHA_DESDE, FECHA_HASTA)
    if rates is not None and len(rates) > 0:
        return rates
    for _ in range(REINTENTOS):
        time.sleep(SLEEP_ENTRE_INTENTOS)
        rates = mt5.copy_rates_range(ticker_real, TIMEFRAME, FECHA_DESDE, FECHA_HASTA)
        if rates is not None and len(rates) > 0:
            return rates
    rates = mt5.copy_rates_from_pos(ticker_real, TIMEFRAME, 0, 1000)
    if rates is None or len(rates) == 0:
        logger.warning(f"Sin datos para {ticker_real} - Error {mt5.last_error()}")
    return rates


def procesar_simbolo(alias_swq: str, meta: dict):
    """Procesa un símbolo: descarga datos, calcula features y guarda CSVs."""
    global preview_counter

    ticker_real = meta["ticker"]
    if not mt5.symbol_select(ticker_real, True):
        codigo, mensaje_mt5 = mt5.last_error()
        motivo = f"no seleccionable (MT5 error {codigo}: {mensaje_mt5})"
        if 'logger' in globals():
            logger.warning("%s descartado: %s", ticker_real, motivo)
        return None, motivo

    info = mt5.symbol_info(ticker_real)
    precio = (info.last or info.bid) if info else None
    if precio is None or precio <= PRECIO_MINIMO:
        motivo = f"precio bajo (último={precio})"
        if 'logger' in globals():
            logger.warning("%s descartado: %s", ticker_real, motivo)
        return None, motivo

    rates = descargar_barras(ticker_real)
    if rates is None:
        motivo = "datos insuficientes (sin barras)"
        if 'logger' in globals():
            logger.warning("%s descartado: %s", ticker_real, motivo)
        return None, motivo
    if len(rates) < MIN_BARRAS:
        motivo = f"datos insuficientes ({len(rates)} < {MIN_BARRAS})"
        if 'logger' in globals():
            logger.warning("%s descartado: %s", ticker_real, motivo)
        return None, motivo

    df = pd.DataFrame(rates)
    df["time"] = pd.to_numeric(df["time"], errors="coerce").astype("Int64")
    df["time"] = pd.to_datetime(df["time"], unit="s", errors="coerce", utc=True).dt.tz_convert(None)
    df = df.dropna(subset=["time"])
    df.set_index("time", inplace=True)
    df = df[~df.index.duplicated(keep="last")].sort_index()

    if not validar_huecos(df.index, HUECO_MAX_DIAS):
        motivo = f"huecos temporales (> {HUECO_MAX_DIAS} días sin datos)"
        if 'logger' in globals():
            logger.warning("%s descartado: %s", ticker_real, motivo)
        return None, motivo

    df = calcular_features(df)
    df["exchange"] = meta.get("exchange", "OTHER")

    file_symbol = safe_symbol_name(alias_swq)
    raw_path = DATA_RAW_DIR / f"{file_symbol}.csv"
    proc_path = DATA_PROCESSED_DIR / f"features_estado_{file_symbol}.csv"
    df.to_csv(raw_path, index_label="time")
    logger.info(f"Datos crudos guardados en {raw_path}")
    df.to_csv(proc_path, index_label="time")
    logger.info(f"Datos procesados guardados en {proc_path}")

    if preview_counter < preview_limit:
        preview_counter += 1
        logger.info(f"📈 {alias_swq} ({ticker_real}) → {len(df)} registros OK.")
        display(df[["open", "high", "low", "close", "tick_volume"]].head(2))
        display(df[["open", "high", "low", "close", "tick_volume"]].tail(2))
    elif preview_counter == preview_limit:
        logger.info("... salida truncada para evitar sobrecarga ...")
        preview_counter += 1

    return df.copy(), None


print("⚙️ Funciones de descarga listas (versión estable).")


⚙️ Funciones de descarga listas (versión estable).


In [9]:
df = calcular_features(df)  # ✅ Genera drawdown, mom_63, volatilidad, score, etc.
df["exchange"] = meta.get("exchange", "OTHER")


NameError: name 'df' is not defined

In [10]:
# 8.-Ejecución paralela (versión estable con barra de progreso) =====
from tqdm import tqdm

print("🚀 Iniciando procesamiento paralelo (modo estable)...")
futuros = {}
total_activos = len(mapping_ext)

# Preparamos barra de progreso
pbar = tqdm(total=total_activos, desc="Procesando activos", ncols=100, colour="cyan")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    for alias, meta in mapping_ext.items():
        futuros[ex.submit(procesar_simbolo, alias, meta)] = alias

for fut in as_completed(futuros):
    alias = futuros[fut]
    try:
        resultado, error = fut.result()
        if error is None:
            precios_consolidados[alias] = resultado["close"].rename(alias)
            activos_ok.append(alias)
        else:
            motivos_falla[error].append(alias)
    except Exception as e:
        err_msg = f"Excepción procesando {alias}: {str(e)}"
        logger.error(err_msg.encode("utf-8", errors="ignore").decode("utf-8", errors="ignore"))
        motivos_falla["excepción"].append(alias)
    finally:
        pbar.update(1)

pbar.close()

print("\n🏁 PROCESAMIENTO FINALIZADO")
print(f"✅ Activos descargados correctamente: {len(activos_ok)} / {total_activos}")
print(f"❌ Fallidos: {sum(len(v) for v in motivos_falla.values())}")
print("🧾 Motivos de falla (resumen):", {k: len(v) for k, v in motivos_falla.items()})


🚀 Iniciando procesamiento paralelo (modo estable)...


Procesando activos:   0%|                                                   | 0/699 [00:00<?, ?it/s]WARNING - AAL descartado: precio bajo (último=12.64)
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\AA.US.csv
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\A.US.csv
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\AAP.US.csv
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\ACHC.US.csv
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\ABBV.US.csv
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\ABT.US.csv
INFO - Datos procesados guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\processed\features_estado_AA.US.csv
INFO - 📈 AA.US (AA) → 750 registros OK.


,open,high,low,close,tick_volume
time,,,,,
2022-11-18,47.91,48.51,46.66,47.31,3540
2022-11-21,45.62,46.88,44.50,46.58,3407


INFO - Datos procesados guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\processed\features_estado_A.US.csv
INFO - Datos procesados guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\processed\features_estado_AAP.US.csv


,open,high,low,close,tick_volume
time,,,,,
2025-11-13,39.84,39.86,37.68,38.14,4547
2025-11-14,37.02,38.13,36.37,37.50,4300


...

,open,high,low,close,tick_volume
time,,,,,
2022-11-18,149.63,149.64,145.56,147.23,1643
2022-11-21,147.85,149.87,146.29,148.04,1777


INFO - ... salida truncada para evitar sobrecarga ...
INFO - ... salida truncada para evitar sobrecarga ...
INFO - ... salida truncada para evitar sobrecarga ...


,open,high,low,close,tick_volume
time,,,,,
2025-11-13,51.26,51.59,50.01,50.20,1204
2025-11-14,49.28,50.07,49.00,49.87,908


WARNING - AES descartado: precio bajo (último=14.02)
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\ACM.US.csv
INFO - Datos procesados guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\processed\features_estado_ACM.US.csv
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\ACN.US.csv
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\ADM.US.csv
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\AFG.US.csv
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\AEP.US.csv
INFO - Datos crudos guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\raw\AFL.US.csv
INFO - Datos procesados guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\data\processed\features_estado_ACN.US.csv
INFO - Datos procesados guardados en C:\Users\User\Downloads\Mini reinanse\notebooks\da


🏁 PROCESAMIENTO FINALIZADO
✅ Activos descargados correctamente: 675 / 699
❌ Fallidos: 24
🧾 Motivos de falla (resumen): {'precio bajo (último=11.13)': 1, 'precio bajo (último=13.56)': 1, 'precio bajo (último=10.59)': 1, 'datos insuficientes (189 < 250)': 1, 'precio bajo (último=13.38)': 1, 'precio bajo (último=14.53)': 1, 'precio bajo (último=14.75)': 1, 'precio bajo (último=11.55)': 1, 'precio bajo (último=14.02)': 1, 'precio bajo (último=10.79)': 1, 'precio bajo (último=5.62)': 1, 'precio bajo (último=14.27)': 1, 'precio bajo (último=8.46)': 1, 'precio bajo (último=11.06)': 1, 'precio bajo (último=3.44)': 1, 'precio bajo (último=8.29)': 1, 'precio bajo (último=8.93)': 1, 'precio bajo (último=10.45)': 1, 'datos insuficientes (143 < 250)': 1, 'precio bajo (último=12.62)': 1, 'precio bajo (último=12.99)': 1, 'precio bajo (último=8.88)': 1, 'precio bajo (último=9.3)': 1, 'precio bajo (último=12.64)': 1}


In [11]:
# 9.- Consolidación de precios descargados =====
print("📦 Consolidando precios descargados...")

if activos_ok:
    df_final = pd.DataFrame(precios_consolidados)
    df_final.index = pd.to_datetime(df_final.index, errors="coerce")
    df_final = df_final[~df_final.index.isna()]
    df_final.index.name = "time"

    # Guardar precios diarios
    diario_path = BACKTESTING_DIR / "prices_df_diario.pkl"
    df_final.to_pickle(diario_path)

    # Versión mensual (último valor de cada mes)
    df_final_mensual = df_final.resample("M").last()
    mensual_path = BACKTESTING_DIR / "prices_df.pkl"
    df_final_mensual.to_pickle(mensual_path)

    print(f"✅ Consolidación completa ({len(df_final.columns)} activos válidos).")
    print(f"🗓️ Fechas: {df_final.index.min().date()} → {df_final.index.max().date()}")
    print(f"💾 Guardado en:\n  - {diario_path}\n  - {mensual_path}")

    # Preview visual
    print("\n🔍 Últimos 5 días de precios (primeras 5 columnas):")
    display(df_final.tail(5).iloc[:, :5])
else:
    print("⚠️ No hay activos válidos para consolidar.")


📦 Consolidando precios descargados...
✅ Consolidación completa (675 activos válidos).
🗓️ Fechas: 2022-11-18 → 2025-11-14
💾 Guardado en:
  - C:\Users\User\Downloads\Mini reinanse\notebooks\backtesting\prices_df_diario.pkl
  - C:\Users\User\Downloads\Mini reinanse\notebooks\backtesting\prices_df.pkl

🔍 Últimos 5 días de precios (primeras 5 columnas):


,PEN.US,HUM.US,ACM.US,ALGN.US,PEGA.US
time,,,,,
2025-11-10,270.41,237.60,130.49,137.90,59.60
2025-11-11,278.77,242.97,131.45,144.14,59.91
2025-11-12,279.48,240.60,132.26,141.49,59.02
2025-11-13,278.47,236.16,133.70,139.24,56.68
2025-11-14,279.49,236.27,133.51,137.15,56.70


In [12]:
# 10.- Estadísticas
total = len(mapping_simple)
exitosos = len(activos_ok)
fallidos = total - exitosos
integridad_pct = round(100 * exitosos / total, 2) if total > 0 else 0.0

por_bolsa = defaultdict(int)
for a in activos_ok:
    por_bolsa[mapping_ext[a]["exchange"]] += 1

print("\n📊 RESUMEN FINAL")
print(f"🧾 Versión: {VERSION}")
print(f"✅ OK: {exitosos} / {total} ({integridad_pct:.2f}%)")
for motivo, lista in motivos_falla.items():
    print(f"   - {motivo}: {len(lista)} activos.")
print("🏷️ Distribución por bolsa:", dict(por_bolsa))

mt5.shutdown()
print("📴 MetaTrader 5 desconectado correctamente.")


📊 RESUMEN FINAL
🧾 Versión: 01_data_v1.4.4_2025-10-20
✅ OK: 675 / 699 (96.57%)
   - precio bajo (último=11.13): 1 activos.
   - precio bajo (último=13.56): 1 activos.
   - precio bajo (último=10.59): 1 activos.
   - datos insuficientes (189 < 250): 1 activos.
   - precio bajo (último=13.38): 1 activos.
   - precio bajo (último=14.53): 1 activos.
   - precio bajo (último=14.75): 1 activos.
   - precio bajo (último=11.55): 1 activos.
   - precio bajo (último=14.02): 1 activos.
   - precio bajo (último=10.79): 1 activos.
   - precio bajo (último=5.62): 1 activos.
   - precio bajo (último=14.27): 1 activos.
   - precio bajo (último=8.46): 1 activos.
   - precio bajo (último=11.06): 1 activos.
   - precio bajo (último=3.44): 1 activos.
   - precio bajo (último=8.29): 1 activos.
   - precio bajo (último=8.93): 1 activos.
   - precio bajo (último=10.45): 1 activos.
   - datos insuficientes (143 < 250): 1 activos.
   - precio bajo (último=12.62): 1 activos.
   - precio bajo (último=12.99): 1 a

In [13]:
# 11.- DIAGNÓSTICO DE INTEGRIDAD DE FEATURES (versión robusta) ===
import pandas as pd

estado_dir = ESTADO_DIR
archivos = sorted(estado_dir.glob('*.csv'))

resumen = []

print('🧪 Iniciando diagnóstico de integridad de features...')
logger.info('Iniciando diagnóstico de integridad de features en %s', estado_dir)

for file in archivos:
    try:
        df = pd.read_csv(file)
        total_rows = len(df)
        valid_cols = df.notna().mean() * 100
        integridad_media = round(valid_cols.mean(), 2)
        simbolo = file.stem.replace('features_estado_', '')
        resumen.append((simbolo, total_rows, integridad_media))
    except Exception as e:
        simbolo = file.stem.replace('features_estado_', '')
        resumen.append((simbolo, 0, 0))
        logger.exception('❌ Error leyendo %s', file)

df_resumen = pd.DataFrame(resumen, columns=['Símbolo', 'Filas', 'Integridad'])
print('🧾 Columnas del resumen:', df_resumen.columns.tolist())

df_resumen = df_resumen.sort_values('Integridad', ascending=False).reset_index(drop=True)
display(df_resumen.head(10))
print('✅ Diagnóstico completado.')


INFO - Iniciando diagnóstico de integridad de features en C:\Users\User\Downloads\Mini reinanse\notebooks\data\estado


🧪 Iniciando diagnóstico de integridad de features...
🧾 Columnas del resumen: ['Símbolo', 'Filas', 'Integridad']


,Símbolo,Filas,Integridad
0,estado_A.US,750,99.96
1,estado_OLED.US,743,99.96
2,estado_NWSA.US,743,99.96
3,estado_NXST.US,743,99.96
4,estado_NYT.US,743,99.96
5,estado_OC.US,743,99.96
6,estado_ODFL.US,743,99.96
7,estado_OGE.US,743,99.96
8,estado_OKE.US,743,99.96
9,estado_OKTA.US,743,99.96


✅ Diagnóstico completado.


In [14]:
# 12.- EXPORTAR ARCHIVOS estado_<symbol>.csv =====
print("💾 Exportando archivos estado_<symbol>.csv a la carpeta /estado...")

exportados = 0

for alias in activos_ok:
    try:
        # Cargar features procesados
        file_symbol = safe_symbol_name(alias)
        proc_path = os.path.join(DATA_PROCESSED_DIR, f"features_estado_{file_symbol}.csv")
        df = pd.read_csv(proc_path, parse_dates=["time"])
        df.set_index("time", inplace=True)

        # Renombrar columnas a lo que espera el modelo
        df_final = pd.DataFrame()
        df_final["retornos"] = df["returns"]
        df_final["equity"] = df["close"]  # Usamos 'close' como proxy para equity

        # Exportar archivo
        output_path = os.path.join(ESTADO_DIR, f"estado_{alias}.csv")
        df_final.to_csv(output_path, index=True)
        print("Felicidades, se ha creado el archivo final exportado con índice, lo cual permite conservar la referencia temporal o identificador único para futuros análisis.")
        exportados += 1
    except Exception as e:
        print(f"❌ Error al exportar {alias}: {e}")

print(f"✅ Archivos exportados correctamente: {exportados} / {len(activos_ok)}")

💾 Exportando archivos estado_<symbol>.csv a la carpeta /estado...
Felicidades, se ha creado el archivo final exportado con índice, lo cual permite conservar la referencia temporal o identificador único para futuros análisis.
Felicidades, se ha creado el archivo final exportado con índice, lo cual permite conservar la referencia temporal o identificador único para futuros análisis.
Felicidades, se ha creado el archivo final exportado con índice, lo cual permite conservar la referencia temporal o identificador único para futuros análisis.
Felicidades, se ha creado el archivo final exportado con índice, lo cual permite conservar la referencia temporal o identificador único para futuros análisis.
Felicidades, se ha creado el archivo final exportado con índice, lo cual permite conservar la referencia temporal o identificador único para futuros análisis.
Felicidades, se ha creado el archivo final exportado con índice, lo cual permite conservar la referencia temporal o identificador único par